https://jira.x5.ru/browse/CVMXC-2583

In [1]:
name = 'CVMXC-2583_rusagro_pa_add'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/02 17:05:52 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

In [5]:
def get_segment_grouped_one_plu(spark, start_date, end_date, plu_codes, guests):
    checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no')
                     )
    
    loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
    loyalty_cardholders = (spark
                            .table(LOYALTY_CARDHOLDERS)
                            .filter(F.col('loyalty_cardholder_age_yrs') >= 18)
                            .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                            .withColumnRenamed("loyalty_account_id", "account_no")
                            .select('account_no', 'customer_rk')
                          )
    clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
    clients_info = clients_info.join(guests, on='customer_rk', how='inner')
    
    checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только доступных гостей
    
    checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                                , 'base_qty'
                               )
                   )
    checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes)) # только чеки с нужными plu
    checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')
    grouped = checks_tc5.groupby(['plu_code']).agg(F.sum('base_qty').alias('qty'))
    
    plu_names = (spark.table(PRODUCTS)
             .withColumnRenamed('plu_id', 'plu_code')
             .select('plu_code', 'plu_nm'))
    seg = grouped.join(plu_names, 'plu_code', 'inner')
    seg_pd = seg.toPandas()
    return seg_pd

### X-27-82-Rusagro-Myaso-Set-sof

In [6]:
camp_id = 'X-27-82-Rusagro-Myaso-Set-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        77974: "sms_viber_tg",
        77974: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-28',
    'date_communication_to': '2022-01-03',

    'wave_ids': [
        78177 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [7]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [8]:
guests = (experiment_guests_communications
          .filter(F.col('group_name') == '0_' +
                str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
          .select('customer_rk').distinct()
         )

In [9]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')

In [10]:
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [11]:
plu_codes = [3017503,3162506,3195297,3444608,3444609,3444610,3495835,3503399,3693488,
                               3695167,3982578,4102644,4105769,4112954,4093072,4175547,4182900,4182902,
                               3666473,3666472,4059048,3940905,3940906,3666497]

In [12]:
seg_pd = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests)

In [13]:
seg_pd['qty'] = seg_pd['qty'].astype('float')

In [14]:
seg_pd

,plu_code,qty,plu_nm
0,4059048,629.31,СЛ.МЯСНИКА Ребра свин.с кор.к.В охл.1кг
1,3666472,"8,028.00",СЛ.МЯСНИКА Шейка св.б/к к.Б п/ф охл.800г
2,4112954,"11,660.00",МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл
3,4105769,"5,917.00",МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 390мл
4,3666497,"1,222.00",СЛ.МЯСНИКА Корейка свин.кат.А охл.800г
5,3495835,736.00,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл
6,3940905,0.91,СЛ.МЯСНИКА Окорок свин.кр/к б/к к.А 1кг
7,3666473,"3,560.00",СЛ.МЯСНИКА Лопатка св.б/к кА п/ф охл800г
8,3195297,"3,605.00",МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл
9,3982578,296.00,МОС.ПРОВАНС.Майонез оливковый 67% 600мл


In [15]:
seg_pd.to_excel(camp_id +'.xlsx', index=False)

### U-27-36_rusagro_meat_viber_sms

In [16]:
camp_id = 'U-27-36_rusagro_meat_viber_sms'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        78301: "sms_viber_tg",
        78298: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-29',
    'date_communication_to': '2022-01-04',

    'wave_ids': [
        78430 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [17]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [18]:
guests = (experiment_guests_communications
          .filter(F.col('group_name') == '0_' +
                str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
          .select('customer_rk').distinct()
         )

In [19]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')

In [20]:
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [21]:
plu_codes = [3017503,3162506,3195297,3444608,3444609,3444610,3495835,3503399,3693488,
                               3695167,3982578,4102644,4105769,4112954,4093072,4175547,4182900,4182902,
                               3666473,3666472,4059048,3940905,3940906,3666497]

In [22]:
seg_pd = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests)

[Stage 23:(3893 + 3) / 3896][Stage 26:(3883 + 13) / 3896][Stage 32:(491 + 1) / 492]6]]]]]22/02/02 17:28:51 ERROR cluster.YarnScheduler: Lost executor 53 on mn-hd0136.x5.ru: Executor heartbeat timed out after 120567 ms
[Stage 30:================================>                    (122 + 78) / 200]]]]22/02/02 17:29:17 ERROR client.TransportResponseHandler: Still have 2 requests outstanding when connection from /192.168.234.56:35454 is closed
22/02/02 17:29:17 ERROR cluster.YarnScheduler: Lost executor 53 on mn-hd0136.x5.ru: Container container_e1121_1643806116192_1007_01_000089 exited from explicit termination request.


In [23]:
seg_pd['qty'] = seg_pd['qty'].astype('float')

In [24]:
seg_pd

,plu_code,qty,plu_nm
0,4059048,375.54,СЛ.МЯСНИКА Ребра свин.с кор.к.В охл.1кг
1,3666472,"3,264.00",СЛ.МЯСНИКА Шейка св.б/к к.Б п/ф охл.800г
2,4112954,"5,535.00",МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл
3,4105769,"6,773.00",МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 390мл
4,3666497,442.00,СЛ.МЯСНИКА Корейка свин.кат.А охл.800г
5,3495835,304.00,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл
6,3940905,5.00,СЛ.МЯСНИКА Окорок свин.кр/к б/к к.А 1кг
7,3666473,"1,477.00",СЛ.МЯСНИКА Лопатка св.б/к кА п/ф охл800г
8,3195297,"1,699.00",МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл
9,3982578,139.00,МОС.ПРОВАНС.Майонез оливковый 67% 600мл


In [25]:
seg_pd.to_excel(camp_id +'.xlsx', index=False)

### X-27-78-Rusagro-Mayonnaise-Saratov-sof

In [93]:
camp_id = 'X-27-78-Rusagro-Mayonnaise-Saratov-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        78196: "sms_viber_tg",
        78203: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-28',
    'date_communication_to': '2022-01-03',

    'wave_ids': [
        78427 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

In [94]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [98]:
guests = experiment_guests_communications.filter(F.col('group_name') == '0_78196_sms_viber_tg').select('customer_rk').distinct()

In [100]:
start_date = datetime.date(2021, 12, 28)
end_date = datetime.date(2022, 1, 3)

In [101]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no')
                     )


In [102]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('loyalty_cardholder_age_yrs') >= 18)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(guests, on='customer_rk', how='inner')

In [103]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только доступных гостей

In [104]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                                , 'base_qty'
                               )
                   )


In [105]:
plu_codes = [3170888,3170890,3170891,3357823,3670061,4082170,4082171,4082172]

In [106]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes)) # только чеки с нужными plu

In [107]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [109]:
grouped = checks_tc5.groupby(['plu_code']).agg(F.sum('base_qty').alias('qty'))

In [110]:
plu_names = (spark.table(PRODUCTS)
             .withColumnRenamed('plu_id', 'plu_code')
             .select('plu_code', 'plu_nm'))

In [111]:
seg = grouped.join(plu_names, 'plu_code', 'inner')

In [112]:
seg_pd2 = seg.toPandas()

In [113]:
seg_pd2

,plu_code,qty,plu_nm
0,4082172,182.000,Я ЛЮБ.ГОТ.Майон.САРАТ.ПРОВ.КЛАС.67%700мл
1,3170890,292.000,САРАТ.ПРОВ.Майонез КЛАССИЧ.67% стак.500г
2,3670061,209.000,САРАТОВ.Майонез ПРОВАНС.клас.67% 250мл
3,3170891,132.000,САРАТ.ПРОВ.Майонез КЛАССИЧ.67% вед.900г
4,4082170,84.000,Я ЛЮБ.ГОТ.Майонез САРАТ.ПРОВ.КЛ.67%200мл
5,4082171,423.000,Я ЛЮБ.ГОТ.Майонез САРАТ.ПРОВ.КЛ.67%400мл
6,3170888,98.000,САРАТ.ПРОВ.Майонез КЛАССИЧ.67% стак.250г


In [114]:
seg_pd2.to_excel(camp_id +'.xlsx', index=False)

### X-27-45-Rusagro-Mayonnaise-Novosibirsk-sof

In [4]:
camp_id = 'X-27-45-Rusagro-Mayonnaise-Novosibirsk-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        76168: "sms_viber_tg",
        76168: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-17',
    'date_communication_to': '2021-12-23',

    'wave_ids': [
        76315 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [5]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [35]:
guests = experiment_guests_communications.filter(F.col('group_name') == '0_76168_sms_viber_cg').select('customer_rk').distinct()

In [36]:
seg_pd3 = get_segment_grouped(spark, 
                    start_date=datetime.date(2021, 12, 17), 
                    end_date=datetime.date(2021, 12, 23), 
                    plu_codes=[4085881,3606802,3606803,3623403,3623402,3606804],
                    guests=guests)

In [40]:
seg_pd3['qty'] = seg_pd3['qty'].astype('float')

In [44]:
seg_pd3

,plu_code,qty,plu_nm
0,4085881,405.00,Я ЛЮБ.ГОТ.Майон.НОВОСИБ.ПРОВ.КЛ.67%200мл
1,3606802,432.00,Майонез НОВОСИБ.ПРОВАНС.КЛАС.67% 400мл
2,3623403,98.00,НОВ.ПРОВ.Майон.ДОМАШ.с доз.55% 700мл
3,3606803,997.00,Майонез НОВОСИБ.ПРОВАНС.КЛАС.67% 700мл


In [45]:
seg_pd3.to_excel(camp_id +'.xlsx', index=False)

### X-27-44-Rusagro-Mayonnaise-Ekaterinburg-sof

In [46]:
camp_id = 'X-27-44-Rusagro-Mayonnaise-Ekaterinburg-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        75929: "sms_viber_tg",
        75929: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-16',
    'date_communication_to': '2021-12-22',

    'wave_ids': [
        76015 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [47]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [48]:
guests = experiment_guests_communications.filter(F.col('group_name') == '0_75929_sms_viber_cg').select('customer_rk').distinct()

In [49]:
seg_pd4 = get_segment_grouped(spark, 
                    start_date=datetime.date(2021, 12, 16), 
                    end_date=datetime.date(2021, 12, 22), 
                    plu_codes=[2146174,3170510,3672055,4009123,4017707,4072721,4072722,4181490],
                    guests=guests)

In [50]:
seg_pd4['qty'] = seg_pd4['qty'].astype('float')

In [51]:
seg_pd4

,plu_code,qty,plu_nm
0,4009123,722.00,ЕЖК Майонез ПРОВАНСАЛЬ 67% д/п 675г
1,3672055,295.00,ЕЖК Майонез ПРОВАНСАЛЬ 67% д/п 193г
2,3170510,"7,087.00",Майонез ПРОВАНСАЛЬ 67% 250г
3,2146174,"2,973.00",Майонез ПРОВАНС.тара полим. 868г
4,4017707,63.00,"ЕЖК Майонез ПРОВАНС.ДОМАШНИЙ 50,5% 672г"
5,4072722,641.00,ЕЖК Соус ПРОВ.ДОМАШНИЙ май.46% д/п 400мл
6,4072721,"3,472.00",ЕЖК Майонез ПРОВАНСАЛЬ 67% д/п 400мл


In [52]:
seg_pd4.to_excel(camp_id +'.xlsx', index=False)

### X-27-18-Rusagro-Competitors-Moscow-sof

In [53]:
camp_id = 'X-27-18-Rusagro-Competitors-Moscow-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        74514: "sms_viber_tg",
        74523: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-08',
    'date_communication_to': '2021-12-13',

    'wave_ids': [
        74719 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [54]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [55]:
guests = experiment_guests_communications.filter(F.col('group_name') == '0_74514_sms_viber_tg').select('customer_rk').distinct()

In [56]:
seg_pd = get_segment_grouped(spark, 
                    start_date=datetime.date(2021, 12, 8), 
                    end_date=datetime.date(2021, 12, 13), 
                    plu_codes=[3017503,3162506,3195297,3444608,3444609,3444610,3495835,3503399,3693488,3695167,
                               3982578,4102644,4105769,4112954,4093072,4175547,4182900,4182902],
                    guests=guests)

In [57]:
seg_pd['qty'] = seg_pd['qty'].astype('float')

In [58]:
seg_pd

,plu_code,qty,plu_nm
0,4112954,67.00,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл
1,4105769,587.00,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 390мл
2,3495835,5.00,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл
3,3195297,41.00,МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл
4,3982578,1.00,МОС.ПРОВАНС.Майонез оливковый 67% 600мл
5,3444609,3.00,МОС.ПРОВАНС.Майонез ОЛИВКОВЫЙ 67% 390мл
6,3444610,4.00,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл
7,3503399,2.00,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 600мл
8,4102644,33.00,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл
9,3695167,2.00,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл


In [59]:
seg_pd.to_excel(camp_id +'.xlsx', index=False)

### X-27-17-Rusagro-HeavyUsers-Moscow-sof

In [60]:
camp_id = 'X-27-17-Rusagro-HeavyUsers-Moscow-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        74389: "sms_viber_tg",
        74389: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-08',
    'date_communication_to': '2021-12-13',

    'wave_ids': [
        74473 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [61]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [62]:
guests = experiment_guests_communications.filter(F.col('group_name') == '0_74389_sms_viber_cg').select('customer_rk').distinct()

In [63]:
seg_pd = get_segment_grouped(spark, 
                    start_date=datetime.date(2021, 12, 8), 
                    end_date=datetime.date(2021, 12, 13), 
                    plu_codes=[3017503,3162506,3195297,3444608,3444609,3444610,3495835,3503399,3693488,
                               3695167,3982578,4102644,4105769,4112954,4093072,4175547,4182900,4182902],
                    guests=guests)

In [64]:
seg_pd['qty'] = seg_pd['qty'].astype('float')

In [65]:
seg_pd

,plu_code,qty,plu_nm
0,4112954,765.00,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл
1,4105769,"7,102.00",МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 390мл
2,3495835,58.00,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл
3,3195297,706.00,МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл
4,3982578,48.00,МОС.ПРОВАНС.Майонез оливковый 67% 600мл
5,3444609,5.00,МОС.ПРОВАНС.Майонез ОЛИВКОВЫЙ 67% 390мл
6,3444610,20.00,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл
7,3503399,34.00,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 600мл
8,4102644,123.00,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл
9,3695167,48.00,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл


In [66]:
seg_pd.to_excel(camp_id +'.xlsx', index=False)

### U-27-5-RussAgro_MoskProvans_viber_sms_gr1

In [67]:
camp_id = 'U-27-5-RussAgro_MoskProvans_viber_sms_gr1'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        74141: "sms_viber_tg",
        74141: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-08',
    'date_communication_to': '2021-12-14',

    'wave_ids': [
        74297 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [68]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [69]:
guests = experiment_guests_communications.filter(F.col('group_name') == '0_' + '74141' + '_sms_viber_cg').select('customer_rk').distinct()

In [70]:
seg_pd = get_segment_grouped(spark, 
                    start_date=datetime.date(2021, 12, 8), 
                    end_date=datetime.date(2021, 12, 14), 
                    plu_codes=[3017503,3162506,3195297,3444608,3444609,3444610,3495835,3503399,3693488,3695167,
                               3982578,4102644,4105769,4112954,4093072,4175547,4182900,4182902],
                    guests=guests)

In [71]:
seg_pd['qty'] = seg_pd['qty'].astype('float')

In [72]:
seg_pd

,plu_code,qty,plu_nm
0,4112954,"1,134.00",МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл
1,4105769,"9,653.00",МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 390мл
2,3495835,99.00,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл
3,3195297,"1,017.00",МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл
4,3982578,47.00,МОС.ПРОВАНС.Майонез оливковый 67% 600мл
5,3444609,8.00,МОС.ПРОВАНС.Майонез ОЛИВКОВЫЙ 67% 390мл
6,3444610,40.00,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл
7,3503399,63.00,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 600мл
8,4102644,104.00,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл
9,3695167,58.00,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл


In [73]:
seg_pd.to_excel(camp_id +'.xlsx', index=False)

### U-27-5-RussAgro_MoskProvans_viber_sms_gr2

In [74]:
camp_id = 'U-27-5-RussAgro_MoskProvans_viber_sms_gr2'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        74102: "sms_viber_tg",
        74102: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-08',
    'date_communication_to': '2021-12-14',

    'wave_ids': [
        74296 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [75]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [76]:
guests = experiment_guests_communications.filter(F.col('group_name') == '0_74102_sms_viber_cg').select('customer_rk').distinct()

In [77]:
seg_pd = get_segment_grouped(spark, 
                    start_date=datetime.date(2021, 12, 8), 
                    end_date=datetime.date(2021, 12, 14), 
                    plu_codes=[3017503,3162506,3195297,3444608,3444609,3444610,3495835,3503399,3693488,3695167,
                               3982578,4102644,4105769,4112954,4093072,4175547,4182900,4182902],
                    guests=guests)

In [78]:
seg_pd['qty'] = seg_pd['qty'].astype('float')

In [79]:
seg_pd

,plu_code,qty,plu_nm
0,4112954,192.00,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл
1,4105769,"2,101.00",МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 390мл
2,3495835,9.00,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл
3,3195297,114.00,МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл
4,3982578,7.00,МОС.ПРОВАНС.Майонез оливковый 67% 600мл
5,3444609,3.00,МОС.ПРОВАНС.Майонез ОЛИВКОВЫЙ 67% 390мл
6,3444610,16.00,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл
7,3503399,13.00,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 600мл
8,4102644,35.00,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл
9,3695167,4.00,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл


In [80]:
seg_pd.to_excel(camp_id +'.xlsx', index=False)